In [1]:
%matplotlib inline

from io import StringIO
from PIL import Image
import imageio
import numpy as np
import pylab
import matplotlib.pyplot as plt
import os

In [34]:
homer_vid = "resources/ex617/hall_monitor_cif.y4m"
resources_path = "resources/ex617"

In [18]:
def QuantizeVid(frames, Q=10):
    frames_q = []
    for frame in frames:
        frames_q.append(np.around(np.around(frame/Q, 2)*Q, 2))
    return frames_q

In [19]:
def QuantizeImage(img, Q=10):
    return np.around(np.around(img/Q, 2)*Q, 2)

In [20]:
def VidConvertGrayScale(rgbVid):
    '''Converts numpy array vid to BW vid'''
    frames_bw = []
    for frame in rgbVid:
        frames_bw.append(np.dot(frame[:,:,:3], [0.2989, 0.587, 0.114]))
    return np.array(frames_bw)

In [21]:
def parse_vid(filename):
    vid = imageio.get_reader(filename,  'ffmpeg')
    fps = vid.get_meta_data()['fps']
    num = 0
    frames = []
    while 1:
        try:
            image = vid.get_data(num)
            frames.append(image)
            num+=1
        except IndexError:
            break

    return np.array(frames), fps

In [22]:
def calc_diff(frames):
    dif_lst = []
    for i in range(1, frames.shape[0]):
        dif_lst.append(frames[i-1,:,:]-frames[i,:,:])
    return np.array(dif_lst)

In [23]:
def comp_vid(frames):
    enc_list= []
    for frame in frames:
        enc_list.append(comp_image(frame))
    return enc_list

In [24]:
def comp_image(frame):
    cnt = 0
    curr_el = frame[0,0]

    encoded = []

    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            if frame[i,j] == curr_el:
                cnt += 1
            else:
                encoded.append('{0}!{1}'.format(curr_el, cnt))
                curr_el = frame[i, j]
                cnt = 1        

    encoded.append('{0}!{1}'.format(curr_el, cnt))
    return encoded

In [25]:
def vid_encode(filename):
    # parse RGB video
    frames, fps = parse_vid(filename)
    # convert to BW
    bw_frames = VidConvertGrayScale(frames)
    # calculate the frame difference matrix
    diffs = calc_diff(bw_frames)
    # quantize the matrix
    q_diffs = QuantizeVid(diffs, Q=100)
    # compress quantized matrix and 1st frame
    q_diffs_comp = comp_vid(q_diffs)
    img1_q = QuantizeImage(bw_frames[0,:,:])
    first_img_comp = comp_image(img1_q)
    
    return first_img_comp, q_diffs_comp, bw_frames[0,:,:].shape, fps


In [26]:
def decode_vid(frames, dims):
    dec_list = []
    for frame in frames:    
        original_dimensions = dims
        decoded = []

        for i in frame:
            symbol, count = i.split('!')
            decoded.extend([float(symbol)]*int(count))   

        dec = np.array(decoded, dtype=np.float64).reshape(original_dimensions)
        dec_list.append(dec)
    return np.array(dec_list)

In [27]:
def decode_image(frame, dims):
    original_dimensions = dims
    decoded = []

    for i in frame:
        symbol, count = i.split('!')
        decoded.extend([float(symbol)]*int(count))   

    decoded = np.array(decoded, dtype=np.float64).reshape(original_dimensions)
    return np.array(decoded)

In [28]:
def reconstruct_vid(img1, diffs):
    final_vid  = [img1]
    cnt = 1
    for frame in diffs:
        final_vid.append(final_vid[-1]-frame)
        
    
    return np.array(final_vid)

In [29]:
def save_vid(frames, path, name, fps):
    writer = imageio.get_writer(os.path.join(path,name), fps=fps,mode="I")

    for frame in frames:
        writer.append_data(frame)
    writer.close()

In [30]:
import pickle

In [33]:
first_img_comp , q_diffs_comp, dims, fps = vid_encode(homer_vid)

#pickle.dump(os.path.join(resources_path,'img1.pckl'), first_img_comp)
#pickle.dump(os.path.join(resources_path,'qdiffs.pckl'), q_diffs_comp)

In [17]:
img1 = decode_image(first_img_comp, dims)
diffs = decode_vid(q_diffs_comp, dims)
tmp2 = reconstruct_vid(img1, diffs)

save_vid(tmp2,resources_path,'hall_final.mp4', fps)

/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [0.0, 255.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-0.4, 255.4]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-0.4, 255.7]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-0.5, 256.2]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-0.5999999999999996, 256.5]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, 

/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-6.0, 261.2]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-5.7, 261.2]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-5.100000000000001, 261.2]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-6.0, 261.3]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-7.0, 261.6]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, 

/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-7.0, 266.8]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-6.0, 266.8]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-6.7, 266.3]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-7.5, 265.8]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-7.300000000000011, 266.8]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, 

/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-8.0, 266.5]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-7.600000000000001, 266.5]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-8.800000000000011, 266.5]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-8.900000000000006, 266.5]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-7.900000000000006, 266.5]
  'range [{2}, {

/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-13.600000000000001, 269.7]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-12.5, 268.8]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-10.5, 270.7]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-10.0, 270.7]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [-8.0, 269.7]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name

In [194]:
first_img_comp

array([[ 1,  1,  1, ..., 25, 25, 25],
       [11, 18, 17, ..., 25, 25, 25],
       [13, 24, 23, ..., 25, 25, 25],
       ...,
       [ 3,  6,  6, ..., 12, 10, 11],
       [ 3,  6,  6, ...,  8,  6,  6],
       [ 3,  7,  7, ...,  1,  0,  0]])